In [6]:
run load_libs.py

<IPython.core.display.Javascript object>

Definition of mean absolute percentage error (MAPE):

\begin{align*}
\mathbb{E}[L]
&= \iint \left|\frac{y(\mathbf{x}) - t}{t} \right| p(\mathbf{x}, t) d\mathbf{x} dt
\end{align*}

Take the derivative wrt. $y(\mathbf{x})$

\begin{align*}
\frac{\partial \mathbb{E}[L]}{\partial y(\mathbf{x})}
= \int_{-\infty}^{y(\mathbf{x})} \left( - \frac{1}{t} \right ) p(\mathbf{x}, t) dt
+ \int_{y(\mathbf{x})}^{0} \frac{1}{t} p(\mathbf{x}, t) dt
+ \int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
+ \int_{y(\mathbf{x})}^{\infty} \left(- \frac{1}{t} \right ) p(\mathbf{x}, t) dt
\end{align*}

If $t > 0$, it simplifies to

\begin{align}
\frac{\partial \mathbb{E}[L]}{\partial y(\mathbf{x})}
= \int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
+ \int_{y(\mathbf{x})}^{\infty} \left(- \frac{1}{t} \right ) p(\mathbf{x}, t) dt
&= 0 \\
\int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
&= \int_{y(\mathbf{x})}^{\infty} \frac{1}{t} p(\mathbf{x}, t) dt \\
\int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
&= \int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
+ \int_{y(\mathbf{x})}^{\infty} \frac{1}{t} p(\mathbf{x}, t) dt
- \int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt \\
2 \int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
&= \mathbb{E}_{F_{t|\mathbf{x}}} \left[ \frac{1}{t} \right] \\
\int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt
&= \frac{1}{2} \mathbb{E}_{F_{t|\mathbf{x}}} \left[ \frac{1}{t} \right]
\end{align}

Note,

* In the 3rd equality, we just plus and minus $\int_{0}^{y(\mathbf{x})} \frac{1}{t} p(\mathbf{x}, t) dt $

Idea: maybe do change of variable $z = \frac{1}{t}$, and apply [inverse distribution](https://en.wikipedia.org/wiki/Inverse_distribution), then it may lead to

\begin{align*}
\int_0^{\frac{1}{y(\mathbf{x})}} z p(\mathbf{x}, z) dz = \frac{1}{2} \mathbb{E}_{F_{z|\mathbf{x}}}[z]
\end{align*}

Then, it leads to

\begin{align*}
\arg \min_{\frac{1}{y(\mathbf{x})}} \mathbb{E}[L]
&= F_{z|\mathbf{x}}(0.5)^{-1} 
\\
\arg \min_{y(\mathbf{x})} \mathbb{E}[L]
&= \frac{1}{F_{z|\mathbf{x}}(0.5)^{-1}} = \frac{1}{F_{\frac{1}{t}|\mathbf{x}}(0.5)^{-1}}
\end{align*}

so, the argmin is the inverse of the median of the distribution of $\frac{1}{t}$ conditioned on $\mathbf{x}$.

# Visualization

In [7]:
dfs = []
for target in [1, 2, 3]:
    ys = np.arange(0.1, 5, 0.01)
    _df = pd.DataFrame({"observed": [target] * len(ys), "predicted": ys}).assign(
        residual=lambda df: df.predicted - df.observed
    )
    dfs.append(
        pd.concat(
            [
                _df.assign(
                    loss=lambda df: df.residual.abs(),
                ).assign(loss_type="MAE"),
                _df.assign(
                    loss=lambda df: (df.residual / df.observed).abs(),
                ).assign(loss_type="MAPE"),
            ]
        )
    )
ndf = pd.concat(dfs).reset_index(drop=True)

In [8]:
alt.Chart(ndf, height=120, width=150).mark_line().encode(
    x="residual", y="loss", color="observed:O"
).facet(facet="loss_type:N")

alt.FacetChart(...)

In [9]:
alt.Chart(ndf.loc[lambda df: df.loss_type == "MAE"], height=120, width=150).mark_line().encode(
    x="residual", y="loss"
).facet(facet="observed:N")

alt.FacetChart(...)

In [10]:
alt.Chart(ndf.loc[lambda df: df.loss_type == "MAPE"], height=120, width=150).mark_line().encode(
    x="residual", y="loss"
).facet(facet="observed:N")

alt.FacetChart(...)